# Enhanced Random Engine

## Add an opening book 

The first improved version of `RandomEngine` 
is the `OpeningRandomEngine`,
which will have access to an opening book. 
(Polyglot)[http://hgm.nubati.net/book_format.html] is a widespread book format for openings.
Because a lot of open-source opening books 
in the `polyglot` format 
can be found on the internet
and because it is supported by the `python-chess` library,
it is used in this example. 
The repository contains some sample `polyglot` opening books.

The `OpeningRandomEngine` 
inherits from `RandomEngine` 
and overwrites the `play` method. 
In the new `play` method, 
the engine first searches for moves 
in the opening book 
using the helper method `getOpeningMoves` 
and,
if no move is found,
falls back to the implementation of `RandomEngine`. 
The method `getOpeningMoves` 
takes the current chessboard as a parameter 
and returns a list of `ScoredMove` objects. 
It uses the `chess.polyglot.MemoryMappedReader` class 
of the `python-chess` library
to open a `polyglot` opening book. 
The `find_all` method of the reader 
is used to search the opening book 
and create a list of scored moves. 

Notice, 
however, 
that the score of the move is defined by the author of the opening book and usually depends on how often and sucessfull it was played by profiecient players. 
In some rare cases, following the book, leads quickly into a draw by fivefold repition, therefore a minimum weight has been defined.

In [ ]:
import random
import chess.polyglot


class OpeningRandomEngine(RandomEngine):
    """Chess engine using opening books to make better moves"""
    MIN_WEIGHT = 50

    def getOpeningMoves(self, board: chess.Board) -> list[ScoredMove]:
        with chess.polyglot.open_reader(
            "../data/polyglot/ProDeo.bin"
        ) as reader:
            nextMoves = [
                ScoredMove(entry.weight, entry.move)
                for entry in reader.find_all(board)
                if entry.weight >= self.MIN_WEIGHT
            ]

        return nextMoves

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        scoredMoves = self.getOpeningMoves(board) or self.analyse(board)
        scoredMove = random.choice(scoredMoves)
        return chess.engine.PlayResult(move=scoredMove.move, ponder=None)

In [ ]:
random.seed(42)
board = playGame(OpeningRandomEngine(), RandomEngine())
print(board.outcome())
board

The `OpeningRandomEngine` 
is not expected to win 
against the `RandomEngine` in general. 
The reason for this is 
the `RandomEngine` often playing bad moves 
such as `Sh6` after `d4`, 
which are usually not included in an opening book. 
The below code shows 
how to calculate the probability 
of randomly hitting the first opening move.

In [ ]:
board = chess.Board()
engine = OpeningRandomEngine()
openingMoves = engine.getOpeningMoves(board)
probabilityFirstMove = len(openingMoves) / len(list(board.legal_moves))
print(
    f"Chance to hit opening move after the first move: {probabilityFirstMove * 100}%"
)

To calculate the probability 
of hitting two moves in a row of the opening book 
by chance
all possible situations after the first opening move have to be examined. 
The script 
takes the average 
of possible opening moves and legal moves 
and uses those to calculate the probability.

In [ ]:
import statistics

legalMovesSecondMove = []
openingMovesSecondMove = []
for scoredMove in openingMoves:
    board.push(scoredMove.move)
    legalMovesSecondMove.append(len(list(board.legal_moves)))
    openingMovesSecondMove.append(len(engine.getOpeningMoves(board)))
    board.pop()

probabilitySecondMove = probabilityFirstMove * (
    statistics.mean(openingMovesSecondMove) /
    statistics.mean(legalMovesSecondMove)
)

print(
    f"Chance to hit opening move two times in a row: {probabilitySecondMove * 100}%"
)

To make sure 
the `OpeningRandomEngine` does only play moves 
from the opening books, 
multiple tests check the engine's choices
against the available opening moves for different seeds.

In [ ]:
engine = OpeningRandomEngine()

# Moves in the ProDeo Opening Book
firstMoves = [
    chess.Move.from_uci("d2d4"),
    chess.Move.from_uci("e2e4"),
    chess.Move.from_uci("c2c4"),
    chess.Move.from_uci("g1f3")
]

random.seed(0)
board = chess.Board()
playResult = engine.play(board)
assert playResult.move in firstMoves, f"Random opening engine chose the move {playResult.move} not in the opening book"

random.seed(42)
board = chess.Board()
playResult = engine.play(board)
assert playResult.move in firstMoves, f"Random opening engine chose the move {playResult.move} not in the opening book"